## Task 2 — Modeling and Tuning (40 points)

In this task, we develop machine learning models to classify univariate ECG time series into one of four classes: normal, atrial fibrillation (AF), other rhythms, and noisy.

The modeling pipeline involves the following key components:
- Designing at least two different model architectures
- Training each model on the training set
- Tuning hyperparameters such as learning rate, number of channels, optimizer, etc.
- Evaluating performance using appropriate metrics (e.g., accuracy, macro F1-score)
- Comparing model performance on the validation set
- Generating predictions on the test data with the best-performing model

We begin with the baseline model suggested in the exercise description:  
A pipeline consisting of STFT → Conv2D → RNN → FC, which first transforms the ECG signals into the frequency domain, processes them with convolutional and recurrent layers, and finally outputs class probabilities through a fully connected layer.

This baseline provides a strong foundation that leverages both frequency and temporal information in the ECG signals.


### ECGDataset: Custom PyTorch Dataset for ECG Signals

We define a custom ECGDataset class to load and serve raw ECG signals for model training. This class inherits from torch.utils.data.Dataset and supports:

- Index-based selection for validation splits
- Conversion of variable-length 1d ECG signals (as numpy arrays) into tensor format
- Association of each signal with its corresponding class label

Since the ECG time series are univariate and vary in length, we keep each sample as an individual 1d tensor rather than padding them in the dataset. Padding will instead be handled dynamically at the batch level using prep_batch.

### Training and Evaluation Functions

We define two utility functions to encapsulate the training and evaluation logic for our model.

train_one_epoch(model, dataloader, optimizer, loss_fn, device)

This function performs one full training pass over the data:
- Puts the model in training mode
- Iterates over batches of data
- Computes the forward pass and loss
- Performs backpropagation and optimizer updates
- Tracks predictions and computes accuracy and macro F1-score at the end

It returns the average loss, accuracy, and F1-score for the epoch.

evaluate(model, dataloader, loss_fn, device)

This function evaluates the model on the validation set:
- Runs in no-grad mode to avoid gradient tracking
- Computes predictions and loss
- Aggregates performance metrics (accuracy and macro F1-score)

This separation ensures clean logging and enables early stopping or validation monitoring during training.

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import torch
import pandas as pd
import numpy as np
import data
from torch.utils.data import DataLoader
from src.parser import read_zip_binary
from src.train import train_model
from src.train_utils import train_one_epoch, evaluate
from src.ecg_dataset import ECGDataset, prep_batch
from src.stft_baseline import BaselineSTFTModel
from sklearn.utils.class_weight import compute_class_weight

# load training data
X_train = read_zip_binary("../data/X_train.zip")

# load training labels
y_train = pd.read_csv("../data/y_train.csv", header=None)
y_train.columns = ["y"]

# load split index
train_idx = np.load("../data/train_idx.npy")
val_idx = np.load("../data/val_idx.npy")

# dataloader
train_dataset = ECGDataset(X_train, y_train, indices=train_idx)
val_dataset = ECGDataset(X_train, y_train, indices=val_idx)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=prep_batch)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=prep_batch)

device = "cpu"

# models is baseline stft model right now
model = BaselineSTFTModel().to(device)

# loss and optimizer

# weights to avoid one class collapse
weights = compute_class_weight(class_weight="balanced", classes=[0, 1, 2, 3], y=y_train["y"])
weights = torch.tensor(weights, dtype=torch.float32).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)

# careful with the learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

# train model
model = train_model(model, train_loader, val_loader, optimizer, loss_fn, device, num_epochs=10)


/Users/cuzdu/Desktop/early-alzhemir-detection/env/lib/python3.12/site-packages/torch/functional.py:704: UserWarning: A window was not provided. A rectangular window will be applied,which is known to cause spectral leakage. Other windows such as torch.hann_window or torch.hamming_window can are recommended to reduce spectral leakage.To suppress this warning and use a rectangular window, explicitly set `window=torch.ones(n_fft, device=<device>)`. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/SpectralOps.cpp:842.)
  return _VF.stft(  # type: ignore[attr-defined]


Unique predictions: (array([0, 1, 2, 3]), array([972,   2, 243,  19]))
Epoch 01 | Time: 52.5s
  Train Loss: 1.3792 | Acc: 0.4368 | F1: 0.2540
  Val   Loss: 1.3693 | Acc: 0.5429 | F1: 0.2506
Unique predictions: (array([0, 1]), array([1002,  234]))
Epoch 02 | Time: 54.7s
  Train Loss: 1.3780 | Acc: 0.4934 | F1: 0.2402
  Val   Loss: 1.3802 | Acc: 0.5113 | F1: 0.2129
Unique predictions: (array([0, 2, 3]), array([973, 245,  18]))
Epoch 03 | Time: 56.2s
  Train Loss: 1.3855 | Acc: 0.5027 | F1: 0.2415
  Val   Loss: 1.3634 | Acc: 0.5477 | F1: 0.2615
Unique predictions: (array([0, 1, 2]), array([965,   2, 269]))
Epoch 04 | Time: 59.4s
  Train Loss: 1.3761 | Acc: 0.5128 | F1: 0.2502
  Val   Loss: 1.3629 | Acc: 0.5453 | F1: 0.2458
Unique predictions: (array([0, 1]), array([985, 251]))
Epoch 05 | Time: 68.6s
  Train Loss: 1.3800 | Acc: 0.5420 | F1: 0.2412
  Val   Loss: 1.3736 | Acc: 0.5024 | F1: 0.2097
Unique predictions: (array([0]), array([1236]))
Epoch 06 | Time: 69.9s
  Train Loss: 1.3962 | Ac